In [ ]:
#!/usr/bin/env python
# coding: utf-8

In [ ]:
from datetime import datetime as dt
import sys
import numpy as np
import os
import pandas as pd
import plotly
from plotly import subplots
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly
import requests
from sklearn.linear_model import LinearRegression

In [ ]:
today_str = " " + dt.now().isoformat()[:16].replace('T', ' ')

In [ ]:
uri = "https://www3.nhk.or.jp/n-data/opendata/coronavirus/"
name = "nhk_news_covid19_prefectures_daily_data.csv"
referer = "https://www3.nhk.or.jp/news/special/coronavirus/data-widget/"
ua = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5)"
ua += " AppleWebKit/537.36 (KHTML, like Gecko)"
ua += " Chrome/67.0.3396.99 Safari/537.36"
headers = {'User-Agent': ua, 'referer': referer}

In [ ]:
response = requests.get(uri + name, headers=headers)
f = open("nhk.csv", "w")
lines = response.text.split("\n")
for line in lines:
    l = line.strip()
    if len(l) > 0:
        f.write(l + "\n")
f.close()

In [ ]:
df = pd.read_csv("nhk.csv", encoding='shift-jis', header=0)

In [ ]:
k_ts         = '日付'
k_pref_code  = '都道府県コード'
k_pref_name  = '都道府県名'
k_case       = '各地の感染者数_1日ごとの発表数'
k_total_case = '各地の感染者数_累計'
k_death      = '各地の死者数_1日ごとの発表数'
k_total_death='各地の死者数_累計'

In [ ]:
latest = df[k_ts].iloc[-1]

In [ ]:
with open("nhk.prev.tmp", "rt") as f:
    prev = f.read().rstrip()
if latest == prev:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()
with open("nhk.prev.tmp", "wt") as f:
    f.write(latest)

In [ ]:
# 文字列を日付型に変換
df[k_ts] = pd.to_datetime(df[k_ts])

In [ ]:
# 都道府県名を取得
prefs = df[k_pref_name].unique()

In [ ]:
def get_template_fig():
    figa = subplots.make_subplots(
        rows=4, cols=6, subplot_titles=prefs[:24],
        horizontal_spacing=0.03, vertical_spacing=0.07)
    figb = subplots.make_subplots(
        rows=4, cols=6, subplot_titles=prefs[24:],
        horizontal_spacing=0.03, vertical_spacing=0.07)
    return figa, figb

In [ ]:
fig1, fig2 = get_template_fig()

In [ ]:
lr = LinearRegression()

In [ ]:
def get_lr_col(v):
    c = 'green'
    if v < -0.5:
        c = 'blue'
    if v >= -0.5:
        c = 'royalblue'
    if v > 0.2:
        c = 'orange'   
    if v > 0.5:
        c = 'red'
    if v < 0.2 and v > -0.2:
        c = 'green'
    return c

In [ ]:
def add_pref2fig(i, x, y, X, fig):
    dfh = df[ df[k_pref_name] == prefs[i]]
    df14 = dfh[-14:]
    Y = df14[k_case].values
    lr.fit(X.reshape(-1, 1), Y.reshape(-1, 1))
    Y_pred = lr.predict(X.reshape(-1, 1)).reshape(1, -1)[0]
    a = lr.coef_[0][0]
    lr_col = get_lr_col(a)
    trace_xy = go.Scatter(
        x=X, y=Y, mode='lines',
        line=dict(width=2, color='white'))
    trace_lr = go.Scatter(
        x=X, y=Y_pred, mode='lines',
        line=dict(width=2, color=lr_col))
    fig.add_trace(trace_xy, y, x)
    fig.add_trace(trace_lr, y, x)

In [ ]:
X = np.arange(14)
for y in np.arange(4):
    for x in np.arange(6):
        i = x + (y * 6)
        add_pref2fig(i, x+1, y+1, X, fig1)
        if i < 23:
            add_pref2fig(i + 24, x+1, y+1, X, fig2)

In [ ]:
def update_axes(fig):
    fig.update_xaxes(
        showgrid=False, showticklabels=False, ticks="",
        zeroline=False)
    fig.update_yaxes(
        showgrid=False, showticklabels=False, ticks="",
        zeroline=False)

In [ ]:
def update_layout(fig, title):
    fig.update_layout(
        margin={"r":5,"t":50,"l":5,"b":5},
        width=600, height=400, template='plotly_dark',
        title=title + " " + today_str, showlegend=False)

In [ ]:
update_axes(fig1)
update_axes(fig2)
twbody1 = "新型コロナ 各都道府県 直近2週間状況1"
update_layout(fig1, twbody1)
twbody2 = "新型コロナ 各都道府県 直近2週間状況2"
update_layout(fig2, twbody2)

In [ ]:
show_and_save_plotly(fig1, "nhk-all-1.jpg", html=False)

In [ ]:
show_and_save_plotly(fig2, "nhk-all-2.jpg", html=False)

In [ ]:
notes = " (縦軸範囲は各都道府県の最大値) "
tw = get_twitter()
tweet_with_image(
    tw, "docs/images/nhk-all-1.jpg", twbody1 + today_str + notes)
tweet_with_image(
    tw, "docs/images/nhk-all-2.jpg", twbody2 + today_str + notes)

In [ ]:
fig3, fig4 = get_template_fig()

In [ ]:
def add_trace2fig(i, x, y, X, fig, key):
    dfh = df[ df[k_pref_name] == prefs[i]]
    Y = dfh[key].values
    trace_xy = go.Scatter(
        x=X, y=Y, mode='lines', line=dict(width=1, color='white'))
    trace_ma = go.Scatter(
        x=X, y=moving_average(Y), mode='lines',
        line=dict(width=.75, color='green'))
    fig.add_trace(trace_xy, y, x)
    fig.add_trace(trace_ma, y, x)

In [ ]:
def make_prefs_subplots(k, figa, figb, X):
    for y in np.arange(4):
        for x in np.arange(6):
            i = x + (y * 6)
            add_trace2fig(i, x+1, y+1, X, figa, k)
            if i < 23:
                add_trace2fig(i + 24, x+1, y+1, X, figb, k)

In [ ]:
xbins = df[k_ts]
make_prefs_subplots(k_case, fig3, fig4, xbins)
update_axes(fig3)
update_axes(fig4)
twbody3 = "新型コロナ 各都道府県 全期間感染状況1"
update_layout(fig3, twbody3)
twbody4 = "新型コロナ 各都道府県 全期間感染状況2"
update_layout(fig4, twbody4)

In [ ]:
show_and_save_plotly(fig3, "nhk-all-3.jpg", html=False)

In [ ]:
show_and_save_plotly(fig4, "nhk-all-4.jpg", html=False)

In [ ]:
tweet_with_image(
    tw, "docs/images/nhk-all-3.jpg", twbody3 + today_str + notes)
tweet_with_image(
    tw, "docs/images/nhk-all-4.jpg", twbody4 + today_str + notes)

In [ ]:
fig5, fig6 = get_template_fig()

In [ ]:
xbins = df[k_ts]
make_prefs_subplots(k_death, fig5, fig6, xbins)
update_axes(fig5)
update_axes(fig6)
twbody5 = "新型コロナ 各都道府県 全期間死者状況1"
update_layout(fig5, twbody5)
twbody6 = "新型コロナ 各都道府県 全期間死者状況2"
update_layout(fig6, twbody6)

In [ ]:
show_and_save_plotly(fig5, "nhk-all-5.jpg", html=False)

In [ ]:
show_and_save_plotly(fig6, "nhk-all-6.jpg", html=False)

In [ ]:
tweet_with_image(
    tw, "docs/images/nhk-all-5.jpg", twbody5 + today_str + notes)
tweet_with_image(
    tw, "docs/images/nhk-all-6.jpg", twbody6 + today_str + notes)

In [ ]:
#df

In [ ]:
#x=df.groupby(k_pref_name).resample('W-Mon', on=k_ts).sum().reset_index().sort_values(by=k_ts)

In [ ]:
#x = x.drop(k_total_case, axis=1)
#x = x.drop(k_total_death, axis=1)
#x['p1w'] = x[k_ts] + pd.Timedelta(days=7)

In [ ]:
#x

In [ ]:
#p = get_populations()

In [ ]:
#pcodes = {}
#ppops = {}
#for k, v in p.items():
#    pcodes[v['ja']] = v['code']
#    ppops[v['ja']] = v['total']

In [ ]:
#x = x.astype({k_case: float})
#x = x.astype({k_death: float})

In [ ]:
#for index, row in x.iterrows():
#    pname = row[k_pref_name].rstrip('県').rstrip('都').rstrip('府')
#    x.at[index, k_pref_code] = pcodes[pname]
#    x.at[index, k_case] = (float(row[k_case]) / ppops[pname]) * 100000
#    x.at[index, k_death] = (float(row[k_death]) / ppops[pname]) * 100000

In [ ]:
#x

In [ ]:
#fig = px.timeline(x[x[k_pref_code] <= 24], x_start=k_ts, x_end='p1w', y=k_pref_code, color=k_case)
#fig.show()

In [ ]:
#fig = px.timeline(x[x[k_pref_code] > 24], x_start=k_ts, x_end='p1w', y=k_pref_code, color=k_case)
#fig.show()